<h1>Chapter 09. Decorators and Closures.</h1>

A Decorator is a callable object that accepts another function as an argument (the function to be decorated).
A Decorator can perform some operations on a function and returns either the function itself or another substitute function or called object.
Decorators are a powerful feature that allows you to modify or extend the behavior of functions or methods without directly modifying their code. They are typically used to add functionality such as logging, caching, or authentication to functions in a modular and reusable way. 

In [1]:
def deco(func):    
    def inner():
        print('Running inner() function')
        
    return inner  # deco() returns its inner() function object


@deco
def target():  # function decorated deco()
    ptint('Running target() function')


# Calling the decorated target() function actually executes inner()
target()

Running inner() function


Inspection shows that `target()` now refers to `inner()`

In [2]:
target

<function __main__.deco.<locals>.inner()>

<h2>When Python executes Decorators</h2>

The main feature of decorators is that they are perfomed as soon as the decorated function is defined. 

In [3]:
# Store references to functions decorated with @register
registry = []


def register(func):
    print(f"Running register({func})")  # print which function is being decorated
    registry.append(func)  # add func to registry
    return func


@register
def f1():
    print('Running f1()')


@register
def f2():
    print('Running f2()')


def f3():
    print('Running f3()')


print(f"Registry -> {registry}")
f1()
f2()
f3()

Running register(<function f1 at 0x10a7fdee0>)
Running register(<function f2 at 0x10a92e160>)
Registry -> [<function f1 at 0x10a7fdee0>, <function f2 at 0x10a92e160>]
Running f1()
Running f2()
Running f3()


<h2>Closures</h2>

A closure is a function, e.g. 'f', with an extended scope that covers variables referenced in the body of 'f', but which are neither global nor local variables of 'f'. Such variables must come from the local scope of the external function enclosing 'f'. Such variables must come from the local scope of the external function enclosing 'f'.

Class-based realization

In [4]:
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return f"The average value of {self.series} is {total / len(self.series)}"

In [5]:
avg = Averager()

avg(10)

'The average value of [10] is 10.0'

In [6]:
avg(11)

'The average value of [10, 11] is 10.5'

In [7]:
avg(12)

'The average value of [10, 11, 12] is 11.0'

Realization using the higher-order function `make_averager`

In [8]:
def make_averager():
    series = []

    # averager closure extends the scope of the functionto include
    # the binding of free variable series
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return f"The average value of {series} is {total / len(series)}"

    return averager

In [9]:
avg = make_averager()

avg(10)

'The average value of [10] is 10.0'

In [10]:
avg(11)

'The average value of [10, 11] is 10.5'

In [11]:
avg(12)

'The average value of [10, 11, 12] is 11.0'

Inspection of the function created by the `make_averager()` function

In [12]:
# Retrieve the variable names
avg.__code__.co_varnames

('new_value', 'total')

In [13]:
# Retrieve the free variable names
avg.__code__.co_freevars

('series',)

In [14]:
# Retrieve the closure
avg.__closure__

(<cell at 0x10a9237f0: list object at 0x10a93a600>,)

<h2><code>nonlocal</code> Announcement</h2>

`nonlocal` is a keyword in Python used within nested functions to indicate that a variable is not local to the innermost function's scope but is in an enclosing (non-global) scope. It allows you to modify variables in the enclosing scope from within the nested function.

Calculating the cumulative average without storing the entire history

In [15]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return f"The average value of {total} is {total / count}."

    return averager

In [16]:
avg = make_averager()

avg(10)

'The average value of 10 is 10.0.'

In [17]:
avg(11)

'The average value of 21 is 10.5.'

In [18]:
avg(12)

'The average value of 33 is 11.0.'

<h2>Realization of a Simple Decorator</h2>

Simple decorator for outputting function execution time

In [19]:
import time


# Define the clock decorator
def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)  # clocked closure includes a free variable 'func'
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f"[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}")
        return result  # return the internal function instead of the decorated function

    return clocked

Using decorator `@clock`

In [20]:
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

In [21]:
print('Calling snooze(.123):')
snooze(.123)

print('Calling factorial(6):')
factorial(6)

Calling snooze(.123):
[0.12633311s] snooze(0.123) -> None
Calling factorial(6):
[0.00000069s] factorial(1) -> 1
[0.00003861s] factorial(2) -> 2
[0.00005643s] factorial(3) -> 6
[0.00007183s] factorial(4) -> 24
[0.00008921s] factorial(5) -> 120
[0.00010648s] factorial(6) -> 720


720

Improved `@clock` decorator

`@functools.wraps()` is a decorator used to copy metadata (like name and docstring) from the original function to a wrapper function, ensuring the wrapped function retains its identity and documentation.

In [22]:
import functools
import time


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f"{k} = {v!r}" for k, v in kwargs.items())
        arg_str = ', '.join(arg_lst)
        print(f"[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}")
        return result

    return clocked

In [23]:
print('Calling snooze(.123):')
snooze(.123)

print('Calling factorial(6):')
factorial(6)

Calling snooze(.123):
[0.12331671s] snooze(0.123) -> None
Calling factorial(6):
[0.00000088s] factorial(1) -> 1
[0.00003072s] factorial(2) -> 2
[0.00004379s] factorial(3) -> 6
[0.00005579s] factorial(4) -> 24
[0.00006889s] factorial(5) -> 120
[0.00008252s] factorial(6) -> 720


720

<h2>Decorators in the Standard Library</h2>

<h3>Memorization with <code>functools.cache</code></h3>

A very overhead recursive way to calculate the `n`th Fibonacci number

In [24]:
@clock
def fibonacci(n):
    if n < 2:
        return n

    return fibonacci(n - 2) + fibonacci(n - 1)


fibonacci(6)

[0.00000062s] fibonacci(0) -> 0
[0.00000152s] fibonacci(1) -> 1
[0.00036264s] fibonacci(2) -> 1
[0.00000040s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000064s] fibonacci(1) -> 1
[0.00002192s] fibonacci(2) -> 1
[0.00004427s] fibonacci(3) -> 2
[0.00042947s] fibonacci(4) -> 3
[0.00000029s] fibonacci(1) -> 1
[0.00000039s] fibonacci(0) -> 0
[0.00000058s] fibonacci(1) -> 1
[0.00002212s] fibonacci(2) -> 1
[0.00004190s] fibonacci(3) -> 2
[0.00000029s] fibonacci(0) -> 0
[0.00000056s] fibonacci(1) -> 1
[0.00002155s] fibonacci(2) -> 1
[0.00000036s] fibonacci(1) -> 1
[0.00000031s] fibonacci(0) -> 0
[0.00000055s] fibonacci(1) -> 1
[0.00002241s] fibonacci(2) -> 1
[0.00004171s] fibonacci(3) -> 2
[0.00008177s] fibonacci(4) -> 3
[0.00014245s] fibonacci(5) -> 5
[0.00059473s] fibonacci(6) -> 8


8

`functools.cache` is a decorator used to cache the results of a function call, storing them in memory to improve performance by avoiding redundant computations for repeated inputs.

Faster realization using caching

In [25]:
import functools


@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n

    return fibonacci(n - 2) + fibonacci(n - 1)


fibonacci(6)

[0.00000045s] fibonacci(0) -> 0
[0.00000129s] fibonacci(1) -> 1
[0.00024390s] fibonacci(2) -> 1
[0.00000112s] fibonacci(3) -> 2
[0.00026428s] fibonacci(4) -> 3
[0.00000073s] fibonacci(5) -> 5
[0.00028290s] fibonacci(6) -> 8


8

<h3>Using <code>functools.lru_cache</code></h3>

`functools.lru_cache` is a decorator used to cache the results of a function call with a least-recently-used (LRU) eviction strategy, which discards the least recently used cache entries when the cache reaches its maximum size. This helps improve performance by storing and reusing previously computed results.

In [26]:
import functools


# Set maximum cache size and enable type-based caching
@functools.lru_cache(maxsize=5, typed=True)
@clock
def fibonacci(n):
    if n < 2:
        return n

    return fibonacci(n - 2) + fibonacci(n - 1)


fibonacci(6)

[0.00000033s] fibonacci(0) -> 0
[0.00000074s] fibonacci(1) -> 1
[0.00008367s] fibonacci(2) -> 1
[0.00000140s] fibonacci(3) -> 2
[0.00010486s] fibonacci(4) -> 3
[0.00000101s] fibonacci(5) -> 5
[0.00012445s] fibonacci(6) -> 8


8

<h3>Function <code>singledispatch</code></h3>

`functools.singledispatch` is a decorator allows you to define a single generic function and then register specialized implementations for different types or classes. This enables function overloading based on the type of arguments passed to the function.

The `htmlize()` function serves as a tool for debugging web applications by generating HTML representations of Python objects of various types. While it accommodates any Python type, it also provides specialized views for certain types to enhance debugging capabilities.
The `singledispatch` decorator creates the `@htmlize.register` function to combine multiple functions into one generalized function. The type of the first argument determines which specialized function is used at runtime. Specialized function names are unimportant, indicated by the use of `_` as the name.

In [27]:
import decimal
import fractions
import html
import numbers

from collections import abc
from functools import singledispatch


# Mark the base function that handles the object type
@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))

    return f"<pre>{content}</pre>"


# Specialize each function with a @htmlize.register decorator
@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br/>\n')

    return f"<p>{content}</p>"


@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)

    return '<ul>\n<li>' + inner + '</li>\n<ul>'


@htmlize.register
def _(n: numbers.Integral) -> str:
    return f"<pre>{n} (0x{n:x})</pre>"


@htmlize.register
def _(n: bool) -> str:
    return f"<pre>{n}</pre>"


@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)

    return f"<pre>{frac.numerator}/{frac.denominator}</pre>"


@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    
    return f"<pre>{x} ({frac.numerator}/{frac.denominator})</pre>"

In [28]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [29]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [30]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br/>\n- a game</p>'

In [31]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [32]:
print(htmlize(['aplha', 66, {3, 2, 1}]))

<ul>
<li><p>aplha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
<ul>


In [33]:
htmlize(True)

'<pre>True</pre>'

In [34]:
htmlize(fractions.Fraction(2, 3))

'<pre>2/3</pre>'

In [35]:
htmlize(2/3)

'<pre>0.6666666666666666 (2/3)</pre>'

In [36]:
htmlize(decimal.Decimal('0.02380952'))

'<pre>0.02380952 (1/42)</pre>'